In [1]:
#import the libraries

from elasticsearch import Elasticsearch
import json
import re
import pandas as pd
import numpy as np
import bs4
import lxml
from bs4 import BeautifulSoup

In [2]:
file = pd.read_csv("casemine.csv")

In [3]:
#initializing the instance of elasticsearch to localhost at port 9200(default)
es = Elasticsearch(HOST="http://localhost", PORT=9200)

In [4]:
#create the index

es.indices.delete(index='judgement_index', ignore=[400, 404])
es.indices.create(index="judgement_index")

C:\Users\Jayant\Anaconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'judgement_index'}

In [5]:
#load the json file

file = open("data_for_index_p_combined.json","r",encoding="utf-8")

data = json.load(file)
file.close()

In [6]:
#insert the data in the index

for i in data.keys():
    es.index(index="judgement_index", doc_type="cases", id=int(i), body=data[i])

C:\Users\Jayant\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  after removing the cwd from sys.path.
C:\Users\Jayant\Anaconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


In [18]:
res = es.get(index="judgement_index", id = 330608)

In [23]:
type(res)

dict

In [24]:
with open("esres.json","w") as outfile:
    json.dump(res,outfile,indent=4)

In [7]:
def get_id(link):
    regex = re.compile(r'[0-9]+')
    result = regex.search(link)
    
    if(result):
        return str(result.group())
    else:
        return ""

In [8]:
def return_similar(docid):
    file = pd.read_csv("casemine.csv")
    
    titles=[]
    
    for i in range(file.shape[0]):
        res = []
        if get_id(str(file.iloc[i,1])) == str(docid):
            res.append(file.iloc[i,2])
            res.append(file.iloc[i,4])
        if get_id(str(file.iloc[i,3])) == str(docid):
            res.append(file.iloc[i,0])
        titles.append(res)
    return titles

In [9]:
def index_func(text_to_search):
    """
    parameters:
    where_to_search: scope of search eg: title, author, etc.
    text_to_search: text that you are looking for in the document
    
    output:
    1. returns, if no output is found
    2. details of judgement, otherwise
    """
    
    #store the result of the query
    body = {
               "_source":[
                  "title",
                  "author",
                   "content"
                    ],
               "size":10000,
                "query":{
                    "multi_match":{
                        "query":      text_to_search,
                          "type":       "phrase",
                          "fields":     [ "author","title","content" ]
                    }
                }
    
        
        }
               
        
    
    result = es.search(index="judgement_index", doc_type="cases", body=body)
    
    #extract the number of results from the result object
    num_results = result['hits']['total']['value']
    
    #return if nothing is found
    if num_results == 0:
        print("No result found!")
        return
    else:
        #print the number of results found
        print(str(num_results)+" result/s found!\n\n")
        
        #print the details of the judgement
        for i in range(num_results):
            res = result['hits']['hits'][i]['_source']
            score = result['hits']['hits'][i]['_score']
            docid = result['hits']['hits'][i]['_id']
            print("RESULT "+str(i+1))
            print("---------")
            print("Document title : "+str(res['title']))
            print("Document author : "+str(res['author']))
            print("Score : "+str(score))
            similar_judgements = return_similar(docid)
            print("SIMILAR JUDGEMENTS")
            for s in similar_judgements:
                if len(s)>1:
                    print(s[0]+str("    Rank : ")+str(s[1]))
                elif len(s)==1:
                    print(s)
            
            print("\n\n\n")
        return result


In [10]:
text_to_search = input("What do you want to search?: ")


res = index_func(text_to_search)

What do you want to search?: Wander


C:\Users\Jayant\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
C:\Users\Jayant\Anaconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in search requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


14 result/s found!


RESULT 1
---------
Document title : Wander Ltd. And Anr. vs Antox India P. Ltd. on 26 April, 1990
Document author : O Venkatachaliah
Score : 4.144434
SIMILAR JUDGEMENTS
Astrazeneca Uk Ltd. v. Orchid Chemicals & Pharmaceuticals Ltd.    Rank : 1
S.M Dyechem Ltd. v. Cadbury (India) Ltd. .    Rank : 2
Veerumal Praveen Kumar v. Needle Industries (India) Ltd. And Anr.    Rank : 3
N.R Dongre And Others v. Whirlpool Corporation And Another    Rank : 4
Pfizer Products Inc. v. Rajesh Chopra    Rank : 5
Mahendra & Mahendra Paper Mills Ltd. v. Mahindra & Mahindra Ltd. .    Rank : 6
Lupin Limited v. Johnson & Johnson    Rank : 7
Rangoon Chemical Works Pvt. Ltd. v. Royal Medicals    Rank : 8
M/S Alkem Laboratories Ltd. Petitioners v. Mega International (P) Ltd. S    Rank : 9
Wockhardt Limited v. Torrent Pharmaceuticals Limited And Another    Rank : 10
Cadila Health Care Ltd. v. Cadila Pharmaceuticals Ltd. .    Rank : 11
Satilila Charitable Society And Anr. v. Skyline Education I

In [16]:
res['hits']['hits'][0]

{'_index': 'judgement_index',
 '_type': 'cases',
 '_id': '330608',
 '_score': 4.144434,
 '_ignored': ['content.keyword'],
 '_source': {'author': 'O Venkatachaliah',
  'title': 'Wander Ltd. And Anr. vs Antox India P. Ltd. on 26 April, 1990',
  'content': 'ORDER\n  \n\nVenkatachaliah, Ojha, J.\n1. Appellants in these appeals by Special Leave, are the Defendants in Civil Suit No. 1220 of 1988 on the original side of the High Court of Madras and assail the order dated 19-1-1990 of the Division Bench of the High Court granting in appeal and in reversal of the order dated 2-3-1989 of the learned single Judge trying the suit an injunction restraining appellants from passing-off their medicinal product Cal-De-Ce as that of the Respondent-Plaintiff, Respondent claimed to have acquired a right in that Trade-mark by continuous user. The temporary injunction was refused by the learned Single-Judge. But in appeals, O.S.A. Nos. 111 and 112 of 1989, preferred against the refusal, the division bench g